# Table Metadata

- SQLAlchemy represents the structure of a relational schema using the concept of **table metadata**.
- SQLAlchemy Core provides this, using a well-known object called `Table` (along with lots of supporting objects). Here is an example:

In [68]:
from sqlalchemy import DateTime, Integer, String, Table, Column, MetaData

metadata = MetaData()

user_account_table = Table(
    "user_account",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("name", String),
    Column("created_at", DateTime),
)

type(user_account_table)

sqlalchemy.sql.schema.Table

## ORM-Centric Table Metadata

- In real world use, most applications are using the ORM to a greater or lesser degree
- Table are constructed **indirectly** using a style known as **Declarative ORM**
- ORM-Centric table metadata integrates very well with typing tools
- Integrates with Python dataclasses too

## ORM-Centric Table Metadata - Declaration

- `DeclarativeBase` is a foundational class in SQLAlchemy 2.0+ that simplifies ORM model definition by providing a declarative style for mapping Python classes to database tables, automatically handling table metadata, and integrating with SQLAlchemy's ORM features.

- `MappedAsDataclass` is a **mixin** that turns SQLAlchemy ORM models into Python dataclasses. It allows you to work with ORM objects more like plain Python dataclasses while retaining SQLAlchemy's ORM capabilities.

We could use both together this way:

In [69]:
from sqlalchemy.orm import DeclarativeBase, MappedAsDataclass


class Base(MappedAsDataclass, DeclarativeBase):
    pass

- When we make subclasses of `Base`, it generates an **ORM mapped class** that will refer to a database table.
- The `MappedAsDataclass` is optional (here more as an example) and indicated that the classes should also be Python dataclasses.

In [70]:
from datetime import datetime
from sqlalchemy.orm import Mapped, mapped_column


class User(Base):  # class can be called whatever we want, no impact on the table name
    __tablename__ = "user_account"

    id: Mapped[int] = mapped_column(
        primary_key=True, init=False
    )  # id field will not be included as a parameter in the __init__ method of the dataclass
    name: Mapped[str]
    full_name: Mapped[str | None]  # this will be nullable at database level
    created_at: Mapped[datetime] = mapped_column(default_factory=datetime.now)

- The `Mapped[]` type indicated to SQLAlchemy that an attribute is matted to a database column. Even if the syntax seems ugly at first, it makes sense because it as a different meaning than the type itself. The class could contain an `int` too which would not nbe mapped to a column at all!
- The `mapped_column()` construct is optional and allow addition details about the database column to be indicated.
- If type of column (e.g. `sqlalchemy.String`) is ommited, type inference is used with type hint to determines the appropriate SQL column type.
- `mapped_column()` not only defines database column mappings but also integrates with Python's dataclass features, allowing you to use arguments that are typically associated with dataclasses.field(), like `default`, `default_factory`, `init`.

## ORM-Centric Table Metadata - The `Mapped` Class

- The `User` class is called a **Declarative Mapped Class**.
- Sice it's also a dataclass, it has methods like default `__init__()` and `__repr__()` based on the directives passed to `mapped_column()`

In [71]:
User("Walt", "Walther White")

User(id=None, name='Walt', full_name='Walther White', created_at=datetime.datetime(2024, 12, 8, 18, 34, 19, 123213))

## ORM-Centric Table Metadata - The Table

- The Declarative Mapped Class set the `Table` object for us, when the class is created. Here is what it looks like:

In [72]:
User.__table__

Table('user_account', MetaData(), Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False), Column('name', String(), table=<user_account>, nullable=False), Column('full_name', String(), table=<user_account>), Column('created_at', DateTime(), table=<user_account>, nullable=False), schema=None)

## ORM-Centric Table Metadata - Emitting DDL

_Note: DDL (Data Definition Language) refers to SQL commands and operations that define or modify the structure of a database. These include commands for creating, altering, and dropping database objects such as tables, indexes, schemas, and constraints._

- Whether we made our table directly or by using ORM Declarative, we can run a `CREATE TABLE` statement using a method `.create_all()`

In [73]:
from sqlalchemy import create_engine

print(Base.metadata.__dict__)

engine = create_engine("sqlite://", echo=True)
with engine.begin() as conn:
    Base.metadata.create_all(conn)  # Create all tables stored in this metadata.

{'tables': FacadeDict({'user_account': Table('user_account', MetaData(), Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False), Column('name', String(), table=<user_account>, nullable=False), Column('full_name', String(), table=<user_account>), Column('created_at', DateTime(), table=<user_account>, nullable=False), schema=None)}), 'schema': None, 'naming_convention': immutabledict({'ix': 'ix_%(column_0_label)s'}), '_schemas': set(), '_sequences': {}, '_fk_memos': defaultdict(<class 'list'>, {})}
2024-12-08 18:34:19,133 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-08 18:34:19,134 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2024-12-08 18:34:19,134 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-08 18:34:19,134 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2024-12-08 18:34:19,135 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-08 18:34:19,135 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (

## ORM-Centric Table Metadata - Foreign Keys


In [ ]:
from sqlalchemy import ForeignKey


class Address(Base):
    __tablename__ = "address"

    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id: Mapped[int] = mapped_column(ForeignKey("user_account.id"))


with engine.begin() as conn:
    Base.metadata.create_all(conn)

2024-12-08 18:34:19,143 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-08 18:34:19,144 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2024-12-08 18:34:19,145 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-08 18:34:19,145 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2024-12-08 18:34:19,146 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-08 18:34:19,147 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2024-12-08 18:34:19,147 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-08 18:34:19,148 INFO sqlalchemy.engine.Engine 
CREATE TABLE address (
	id INTEGER NOT NULL, 
	email_address VARCHAR NOT NULL, 
	user_id INTEGER NOT NULL, 
	PRIMARY KEY (id), 
	FOREIGN KEY(user_id) REFERENCES user_account (id)
)


2024-12-08 18:34:19,148 INFO sqlalchemy.engine.Engine [no key 0.00040s] ()
2024-12-08 18:34:19,149 INFO sqlalchemy.engine.Engine COMMIT
